In [1]:
import numpy as np

In [11]:
class Function:
    def __init__(self, x = None, n = 2, lb = np.array([-5, -5]), ub = np.array([5, 5])):
        self.n_x = n
        if x is not None:
            assert(x.shape[0] == self.n_x)
            self.fvalue = self.getFValue(x)
        self.x = x
        self.fvalue = None
        assert(lb.shape[0] == self.n_x)
        self.lb = lb
        assert(ub.shape[0] == self.n_x)
        self.ub = ub
    
    def isFeasible(self, x):
        return np.all(x >= self.lb) and np.all(x <= self.ub)
    
    def getFValue(self, x):
        func_value = 4*x[0]**2 - 2.1*x[0]**4 + (x[0]**6)/3 + x[0]*x[1] - 4*x[1]**2 + 4*x[1]**4
        return func_value
    
    def initRandomSoln(self):
        self.x = np.random.rand(self.n_x) * (self.ub - self.lb) + self.lb
        assert(self.isFeasible(self.x))
        self.fvalue = self.getFValue(self.x)
    
    def getNeighbourSoln(self):
        r = np.random.rand(self.n_x)
        x_new = self.x + r * (self.ub - self.x) + (1 - r) * (self.lb - self.x)
        assert(self.isFeasible(x_new))
        return x_new

In [22]:
class SimulatedAnnealing:
    def __init__(self, problem, max_iter = 100, init_temp = 100, final_temp = 1e-03, iter_per_temp = 5, cooling_schedule = "linear", beta = 5, alpha = 0.9):
        self.problem = problem
        self.max_iter = max_iter
        self.init_temp = init_temp
        self.final_temp = final_temp
        self.iter_per_temp = iter_per_temp
        self.cooling_schedule = cooling_schedule
        self.beta = min(beta, (init_temp - final_temp)/max_iter)
        self.alpha = alpha
        self.curr_temp = None
        self.sols = None
        self.fvalues = None
        self.best_sol = None
        self.best_fvalue = None
    
    def cool_down_temp(self, curr_iter):
        schedules = {
            "linear": self.curr_temp - self.beta,
            "geometric": self.curr_temp * self.alpha,
            "logarithmic": self.curr_temp / np.log(curr_iter+1),
            "exponential": self.curr_temp / (1 + self.beta*self.curr_temp),
            "hybrid": (curr_iter/(curr_iter+1)) if curr_iter <= 0.5*self.max_iter else (self.init_temp*(self.alpha**curr_iter))
        }
        return schedules.get(self.cooling_schedule, schedules.get("linear"))

    def perform_algorithm(self):
        self.problem.initRandomSoln()
        self.curr_temp = self.init_temp
        self.sols = [self.problem.x]
        self.fvalues = [self.problem.fvalue]
        self.best_sol = self.problem.x
        self.best_fvalue = self.problem.fvalue
        for iter in range(self.max_iter):
            for _ in range(self.iter_per_temp):
                sol_neighbour = self.problem.getNeighbourSoln()
                fvalue_neighbour = self.problem.getFValue(sol_neighbour)
                if fvalue_neighbour < self.fvalues[-1]:
                    # neighbour is better and is accpeted
                    self.sols.append(sol_neighbour)
                    self.fvalues.append(fvalue_neighbour)
                else:
                    p = np.exp(-(fvalue_neighbour - self.fvalues[-1]) / self.curr_temp)
                    if np.random.rand() < p:
                        # neighbour is worse and is accepted according to probability
                        self.sols.append(sol_neighbour)
                        self.fvalues.append(fvalue_neighbour)
                    else:
                        # neighbour is worse and is rejected
                        self.sols.append(self.sols[-1])
                        self.fvalues.append(self.fvalues[-1])
                # update best solution reached so far
                if self.fvalues[-1] < self.best_fvalue:
                    self.best_sol = self.sols[-1]
                    self.best_fvalue = self.fvalues[-1]
            # update temperature
            self.curr_temp = self.cool_down_temp(iter)
    
    def visualize(self):
        pass


In [26]:
problem = Function()
SA = SimulatedAnnealing(problem, max_iter=200, init_temp=100, final_temp=1e-03, iter_per_temp=10, cooling_schedule="hybrid", alpha=0.85)
SA.perform_algorithm()
print(SA.best_sol)
print(SA.best_fvalue)

[-0.14668465  0.72586298]
-1.018486790432261


C:\Users\shaim\AppData\Local\Temp/ipykernel_7100/1154490701.py:21: RuntimeWarning: divide by zero encountered in double_scalars
  "logarithmic": self.curr_temp / np.log(curr_iter+1),
C:\Users\shaim\AppData\Local\Temp/ipykernel_7100/1154490701.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  p = np.exp(-(fvalue_neighbour - self.fvalues[-1]) / self.curr_temp)
